In [1]:
import os
import datetime
import time

import tensorflow as tf
from tensorflow import keras

import resnet

In [2]:
NUM_GPUS = 1
BS_PER_GPU = 64
NUM_EPOCHS = 50

HEIGHT = 32
WIDTH = 32
NUM_CHANNELS = 3
NUM_CLASSES = 10
NUM_TRAIN_SAMPLES = 50000

BASE_LEARNING_RATE = 0.1
LR_SCHEDULE = [(0.1, 30), (0.01, 45)]

In [3]:
modelsDir = 'models'
resultsDir = 'results'
logsDir = 'logs'

In [4]:
def normalize(x, y):
  x = tf.image.per_image_standardization(x)
  return x, y


def augmentation(x, y):
    x = tf.image.resize_with_crop_or_pad(
        x, HEIGHT + 8, WIDTH + 8)
    x = tf.image.random_crop(x, [HEIGHT, WIDTH, NUM_CHANNELS])
    x = tf.image.random_flip_left_right(x)
    return x, y	


def schedule(epoch):
  initial_learning_rate = BASE_LEARNING_RATE * BS_PER_GPU / 128
  learning_rate = initial_learning_rate
  for mult, start_epoch in LR_SCHEDULE:
    if epoch >= start_epoch:
      learning_rate = initial_learning_rate * mult
    else:
      break
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate


In [ ]:
(x,y), (x_test, y_test) = keras.datasets.cifar10.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((x,y))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

tf.random.set_seed(22)
train_dataset = train_dataset.map(augmentation).map(normalize).shuffle(NUM_TRAIN_SAMPLES).batch(BS_PER_GPU * NUM_GPUS, drop_remainder=True)
test_dataset = test_dataset.map(normalize).batch(BS_PER_GPU * NUM_GPUS, drop_remainder=True)


input_shape = (HEIGHT, WIDTH, NUM_CHANNELS)
img_input = tf.keras.layers.Input(shape=input_shape)
opt = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

if NUM_GPUS == 1:
    model = resnet.resnet56(img_input=img_input, classes=NUM_CLASSES)
    model.compile(
              optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
else:
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
      model = resnet.resnet56(img_input=img_input, classes=NUM_CLASSES)
      model.compile(
                optimizer=opt,
                loss='sparse_categorical_crossentropy',
                metrics=['sparse_categorical_accuracy'])  

In [6]:
print(x.shape, y.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [7]:
logs = os.path.join(logsDir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

file_writer = tf.summary.create_file_writer(logs + "/metrics")
file_writer.set_as_default()

tensorboard_callback = keras.callbacks.TensorBoard(
  log_dir=logs,
#  update_freq='epoch', #'batch'
  histogram_freq=1)

lr_schedule_callback = keras.callbacks.LearningRateScheduler(schedule)

startTime = time.time()

model.fit(train_dataset,
          epochs=NUM_EPOCHS,
          validation_data=test_dataset,
          validation_freq=1,
          callbacks=[tensorboard_callback, lr_schedule_callback])

endTime = time.time()

elapsedTime = (endTime - startTime)/60.
print("\nTotal time for model training: {} minutes".format(elapsedTime))

Epoch 1/50
  1/781 [..............................] - ETA: 0s - loss: 5.6268 - sparse_categorical_accuracy: 0.0312WARNING:tensorflow:From /data/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
781/781 [==============================] - 29s 37ms/step - loss: 2.7804 - sparse_categorical_accuracy: 0.2675 - val_loss: 2.9466 - val_sparse_categorical_accuracy: 0.2429
Epoch 2/50
781/781 [==============================] - 27s 34ms/step - loss: 2.1365 - sparse_categorical_accuracy: 0.4437 - val_loss: 3.1172 - val_sparse_categorical_accuracy: 0.3055
Epoch 3/50
781/781 [==============================] - 27s 34ms/step - loss: 1.7540 - sparse_categorical_accuracy: 0.5506 - val_loss: 2.7139 - val_sparse_categorical_accuracy: 0.3427
Epoch 4/50
781/781 [==============================] - 27s 3

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



781/781 [==============================] - 27s 34ms/step - loss: 0.3456 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.3467 - val_sparse_categorical_accuracy: 0.7845
Epoch 48/50
781/781 [==============================] - 27s 34ms/step - loss: 0.3447 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.3454 - val_sparse_categorical_accuracy: 0.7842
Epoch 49/50
781/781 [==============================] - 27s 34ms/step - loss: 0.3435 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.3458 - val_sparse_categorical_accuracy: 0.7847
Epoch 50/50
781/781 [==============================] - 27s 34ms/step - loss: 0.3426 - sparse_categorical_accuracy: 0.9999 - val_loss: 1.3450 - val_sparse_categorical_accuracy: 0.7836

Total time for model training: 26.860927093029023 minutes


model.evaluate(test_dataset)

model.save('model.h5')

new_model = keras.models.load_model('model.h5')
 
new_model.evaluate(test_dataset)